# Extracting some the image statistics from the DRIVE database used in the study

In [1]:
# importing the necessary packages

import glob
import numpy as np
import pandas as pd
from PIL import Image, ImageFilter
import os.path

from config import drive_dir, image_stats_file

In [2]:
# loading the list of files
files= glob.glob(os.path.join(drive_dir, '*', '*', '*'))

In [3]:
# splitting the training and test images
training= [f for f in files if 'training' in f]
test= [f for f in files if 'test' in f]

In [4]:
# splitting images, fov masks and manual segmentations

training_img= [f for f in training if 'images' in f]
training_mask= [f for f in training if 'mask' in f]
training_manual= [f for f in training if 'manual' in f]

test_img= [f for f in test if 'images' in f]
test_mask= [f for f in test if 'mask' in f]
test_manual1= [f for f in test if 'manual1' in f]
test_manual2= [f for f in test if 'manual2' in f]

In [5]:
# extracting image statistics:
# * number of positive and negative pixels for both the training and test images, with and without FoV
# * computing the performance scores by treating the annotations of observer 1 as the ground truth and those of observer 2 as segmentations and vice versa

img_stats= []

for img in training_img:
    # calculating basic statistics for the training images
    identifier= img.split(os.sep)[-1].split('_')[0]
    mask= [f for f in training_mask if identifier in f][0]
    manual= [f for f in training_manual if identifier in f][0]

    mask= np.array(Image.open(mask))
    manual= np.array(Image.open(manual))

    p_no_fov= np.sum(manual > 0)
    n_no_fov= np.sum(manual == 0)
    p_fov= np.sum(np.logical_and(manual > 0, mask > 0))
    n_fov= np.sum(np.logical_and(manual == 0, mask > 0))
    size_fov= np.sum(mask > 0)
    n= np.prod(manual.shape)

    img_stats.append([identifier, size_fov, p_fov, n_fov, True, 1, None, None, None, None, None, mask.shape[0], mask.shape[1], np.prod(mask.shape), False])
    img_stats.append([identifier, n, p_no_fov, n_no_fov, False, 1, None, None, None, None, None, mask.shape[0], mask.shape[1], np.prod(mask.shape), False])

for img in test_img:
    # calculating basic statistics and performance scores for the test images
    identifier= img.split(os.sep)[-1].split('_')[0]
    mask= [f for f in test_mask if identifier in f][0]
    manual1= [f for f in test_manual1 if identifier in f][0]
    manual2= [f for f in test_manual2 if identifier in f][0]

    mask= np.array(Image.open(mask))
    manual1= np.array(Image.open(manual1))
    manual2= np.array(Image.open(manual2))

    p_no_fov= np.sum(manual1 > 0)
    n_no_fov= np.sum(manual1 == 0)
    p_fov= np.sum(np.logical_and(manual1 > 0, mask > 0))
    n_fov= np.sum(np.logical_and(manual1 == 0, mask > 0))
    size_fov= np.sum(mask > 0)
    n= np.prod(manual1.shape)

    # manual1 being the ground truth, treating manual2 as a segmentation, using FOV
    tp= np.sum(np.logical_and(np.logical_and(manual1 > 0, manual2 > 0), mask > 0))
    fp= np.sum(np.logical_and(np.logical_and(manual1 == 0, manual2 > 0), mask > 0))
    tn= np.sum(np.logical_and(np.logical_and(manual1 == 0, manual2 == 0), mask > 0))
    fn= np.sum(np.logical_and(np.logical_and(manual1 > 0, manual2 == 0), mask > 0))

    img_stats.append([identifier, size_fov, p_fov, n_fov, True, 1, tp, fp, tn, fn, 1, mask.shape[0], mask.shape[1], np.prod(mask.shape), True])

    # manual1 being the ground truth, treating manual2 as a segmentation, without FOV
    tp= np.sum(np.logical_and(manual1 > 0, manual2 > 0))
    fp= np.sum(np.logical_and(manual1 == 0, manual2 > 0))
    tn= np.sum(np.logical_and(manual1 == 0, manual2 == 0))
    fn= np.sum(np.logical_and(manual1 > 0, manual2 == 0))

    img_stats.append([identifier, n, p_no_fov, n_no_fov, False, 1, tp, fp, tn, fn, 1, mask.shape[0], mask.shape[1], np.prod(mask.shape), True])

    # manual2 being the ground truth, treating manual1 as a segmentation, using FOV
    p_no_fov= np.sum(manual2 > 0)
    n_no_fov= np.sum(manual2 == 0)
    p_fov= np.sum(np.logical_and(manual2 > 0, mask > 0))
    n_fov= np.sum(np.logical_and(manual2 == 0, mask > 0))
    size_fov= np.sum(mask > 0)
    n= np.prod(manual2.shape)

    tp= np.sum(np.logical_and(np.logical_and(manual2 > 0, manual1 > 0), mask > 0))
    fp= np.sum(np.logical_and(np.logical_and(manual2 == 0, manual1 > 0), mask > 0))
    tn= np.sum(np.logical_and(np.logical_and(manual2 == 0, manual1 == 0), mask > 0))
    fn= np.sum(np.logical_and(np.logical_and(manual2 > 0, manual1 == 0), mask > 0))

    img_stats.append([identifier, size_fov, p_fov, n_fov, True, 2, tp, fp, tn, fn, 2, mask.shape[0], mask.shape[1], np.prod(mask.shape), True])

    # manual2 being the ground truth, treating manual1 as a segmentation, without FOV
    tp= np.sum(np.logical_and(manual2 > 0, manual1 > 0))
    fp= np.sum(np.logical_and(manual2 == 0, manual1 > 0))
    tn= np.sum(np.logical_and(manual2 == 0, manual1 == 0))
    fn= np.sum(np.logical_and(manual2 > 0, manual1 == 0))

    img_stats.append([identifier, n, p_no_fov, n_no_fov, False, 2, tp, fp, tn, fn, 2, mask.shape[0], mask.shape[1], np.prod(mask.shape), True])

# constructing the image statistics dataframe
img_stats= pd.DataFrame(img_stats, columns= ['id', 'n_all', 'p', 'n', 'fov', 'annotator', 'tp', 'fp', 'tn', 'fn', 'ground_truth', 'width', 'height', 'img_size', 'test'])


In [6]:
# computing the image level accuracy, sensitivity and specificity scores
img_stats['acc']= (img_stats['tp'] + img_stats['tn'])/(img_stats['n_all'])
img_stats['sens']= (img_stats['tp'])/(img_stats['p'])
img_stats['spec']= (img_stats['tn'])/(img_stats['n'])
img_stats['dice']= (2*img_stats['tp'])/(2*img_stats['tp'] + img_stats['fp'] + img_stats['fn'])
img_stats['dice_no_fp']= (2*img_stats['tp'])/(2*img_stats['tp'] + img_stats['fn'])

In [7]:
img_stats

,id,n_all,p,n,fov,annotator,tp,fp,tn,fn,ground_truth,width,height,img_size,test,acc,sens,spec,dice,dice_no_fp
0,34,226542,32287,194255,True,1,NaN,NaN,NaN,NaN,NaN,584,565,329960,False,NaN,NaN,NaN,NaN,NaN
1,34,329960,32287,297673,False,1,NaN,NaN,NaN,NaN,NaN,584,565,329960,False,NaN,NaN,NaN,NaN,NaN
2,24,227726,38215,189511,True,1,NaN,NaN,NaN,NaN,NaN,584,565,329960,False,NaN,NaN,NaN,NaN,NaN
3,24,329960,38229,291731,False,1,NaN,NaN,NaN,NaN,NaN,584,565,329960,False,NaN,NaN,NaN,NaN,NaN
4,29,227309,27738,199571,True,1,NaN,NaN,NaN,NaN,NaN,584,565,329960,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,18,329960,30321,299639,False,2,22439.0,3705.0,295934.0,7882.0,2.0,584,565,329960,True,0.964884,0.740048,0.987635,0.794793,0.850607
116,15,227394,23612,203782,True,1,18896.0,5720.0,198062.0,4716.0,1.0,584,565,329960,True,0.954106,0.800271,0.971931,0.783611,0.889056
117,15,329960,23614,306346,False,1,18896.0,5720.0,300626.0,4718.0,1.0,584,565,329960,True,0.968366,0.800203,0.981328,0.783579,0.889014
118,15,227394,24616,202778,True,2,18896.0,4716.0,198062.0,5720.0,2.0,584,565,329960,True,0.954106,0.767631,0.976743,0.783611,0.868542


In [8]:
# writing the results into a csv file
img_stats.to_csv(image_stats_file, index=False, header=True)

In [9]:
# mean fov size
entire_img_size= img_stats[img_stats['fov'] == False]['n_all'].reset_index(drop=True).astype(float)
fov_size= img_stats[img_stats['fov'] == True]['n_all'].reset_index(drop=True).astype(float)
np.mean((entire_img_size - fov_size)/entire_img_size)

0.31217460702307354

In [10]:
# filtering for test images with performance scores using annotation #1 as ground truth, with and without FoV
img_stats_test= img_stats[(img_stats['test'] == True) & (img_stats['ground_truth'] == 1)]
with_fov= img_stats_test[img_stats_test['fov'] == True].reset_index(drop=True)
without_fov= img_stats_test[img_stats_test['fov'] == False].reset_index(drop=True)

In [11]:
# determining the mean scores treating the annotations of observer 2 as a segmentation under the FoV
np.mean(with_fov[['p', 'n', 'acc', 'sens', 'spec', 'dice', 'dice_no_fp']])

p              28882.450000
n             198024.700000
acc                0.947283
sens               0.776027
spec               0.972495
dice               0.788123
dice_no_fp         0.872700
dtype: float64

In [12]:
# determining the standard deviation of scores treating the annotations of observer 2 as a segmentation under the FoV
np.std(with_fov[['p', 'n', 'acc', 'sens', 'spec', 'dice', 'dice_no_fp']])

p             2679.699134
n             3089.591674
acc              0.004720
sens             0.057926
spec             0.008102
dice             0.020045
dice_no_fp       0.036558
dtype: float64

In [13]:
# determining the mean scores treating the annotations of observer 2 as a segmentation without the FoV
np.mean(without_fov[['acc', 'sens', 'spec', 'dice']])

acc     0.963703
sens    0.775673
spec    0.981897
dice    0.787928
dtype: float64

In [14]:
# determining the maximum difference in the number of positives with and without FoV
np.max(without_fov['p'].reset_index(drop=True) - with_fov['p'].reset_index(drop=True))

78

In [15]:
# determining the ratio of positives and negatives when FoV is used
np.mean(with_fov['p']/with_fov['n'])

0.14608756243826987